In [1]:
import pandas as pd
from google.cloud import bigquery as bq

In [2]:
sigDMD = pd.read_table("./preliminary/sig-DMD.union.includeERR.reference", header=None)
sigDMD.columns = ['chrom', 'start', 'end', 'sig-DMD', 'DMV', 'pairs', 'numberOfPairs']
sigDMD.head()

,chrom,start,end,sig-DMD,DMV,pairs,numberOfPairs
0,chr1,862600,865000,sig-DMD_1,DMV_4,"nontrip_trip,normal_trip",2
1,chr1,930000,934000,sig-DMD_2,DMV_6,"nontrip_trip,normal_nontrip,normal_trip",3
2,chr1,1209000,1211000,sig-DMD_3,DMV_7,"normal_nontrip,normal_trip",2
3,chr1,1286600,1296000,sig-DMD_4,DMV_9,"ers_err,normal_nontrip,normal_trip",3
4,chr1,1365000,1369600,sig-DMD_5,DMV_11,"ers_err,normal_nontrip,normal_trip",3


In [3]:
sigDMD.shape #4375 signatures

(4375, 7)

In [4]:
client = bq.Client()
hm450_anno_ref = 'isb-cgc.platform_reference.methylation_annotation'

In [5]:
query = """\
SELECT 
    CONCAT("chr",CHR) AS chrom, 
    (MAPINFO-1) AS start, (MAPINFO+1) AS `end`, 
    IlmnID AS probe_id, UCSC
FROM
    `{}`
WHERE
    CHR IS NOT NULL
    AND MAPINFO IS NOT NULL
ORDER BY 
    CHR, MAPINFO
ASC
""".format(hm450_anno_ref)

In [6]:
hm450_anno = client.query(query).to_dataframe()

print(hm450_anno.shape)

hm450_anno.head()

(485512, 5)


,chrom,start,end,probe_id,UCSC
0,chr1,15864,15866,cg13869341,"[{'RefGene_Group': 'Body', 'RefGene_Accession'..."
1,chr1,18826,18828,cg14008030,"[{'RefGene_Group': 'Body', 'RefGene_Accession'..."
2,chr1,29406,29408,cg12045430,"[{'RefGene_Group': 'TSS200', 'RefGene_Accessio..."
3,chr1,29424,29426,cg20826792,"[{'RefGene_Group': 'TSS200', 'RefGene_Accessio..."
4,chr1,29434,29436,cg00381604,"[{'RefGene_Group': 'TSS200', 'RefGene_Accessio..."


In [7]:
print(hm450_anno.iloc[0,3])
type(hm450_anno.iloc[0,3])

cg13869341


str

In [8]:
type(hm450_anno.iloc[0,3][0])

str

In [9]:
RefGene_Group=[] 
RefGene_Accession=[] 
RefGene_Name=[]

def transform(l):
    d=l[0]
    global RefGene_Group, RefGene_Accession, RefGene_Name
    RefGene_Group.append(d['RefGene_Group'])
    RefGene_Accession.append(d['RefGene_Accession'])
    RefGene_Name.append(d['RefGene_Name'])

In [10]:
hm450_anno['UCSC'].apply(transform)

print(len(RefGene_Group))
print(len(RefGene_Accession))
print(len(RefGene_Name))

485512
485512
485512


In [11]:
hm450_anno.drop(columns=['UCSC'], inplace=True)
hm450_anno['UCSC.RefGene_Group'] = RefGene_Group
hm450_anno['UCSC.RefGene_Accession'] = RefGene_Accession
hm450_anno['UCSC.RefGene_Name'] = RefGene_Name

hm450_anno.head()

,chrom,start,end,probe_id,UCSC.RefGene_Group,UCSC.RefGene_Accession,UCSC.RefGene_Name
0,chr1,15864,15866,cg13869341,Body,NR_024540,WASH5P
1,chr1,18826,18828,cg14008030,Body,NR_024540,WASH5P
2,chr1,29406,29408,cg12045430,TSS200,NR_024540,WASH5P
3,chr1,29424,29426,cg20826792,TSS200,NR_024540,WASH5P
4,chr1,29434,29436,cg00381604,TSS200,NR_024540,WASH5P


In [12]:
hm450_anno.to_csv("./metadata/hm450_annotation", sep="\t", header=None, index=False)

In [13]:
%%bash 
sort -k1,1 -k2,2n datasets/hm450_annotation > /tmp/temp.bed

mv /tmp/temp.bed datasets/hm450_annotation

bedtools intersect -wa -wb \
-a preliminary/sig-DMD.union.includeERR.reference \
-b metadata/hm450_annotation | \
awk '{OFS="\t"} {print $8,$9,$10,$11,$12,$13,$14,$4,$5,$6,$7}' > \
metadata/sig-DMD.union.includeERR.hm450probes

In [14]:
! wc -l metadata/sig-DMD.union.includeERR.hm450probes

25329 datasets/sig-DMD.union.includeERR.hm450probes


In [15]:
! head metadata/sig-DMD.union.includeERR.hm450probes

chr1	864878	864880	cg02896266	Body	NM_152486	SAMD11	sig-DMD_1	DMV_4	nontrip_trip,normal_trip	2
chr1	931326	931328	cg03648020				sig-DMD_2	DMV_6	nontrip_trip,normal_nontrip,normal_trip	3
chr1	933305	933307	cg01729262				sig-DMD_2	DMV_6	nontrip_trip,normal_nontrip,normal_trip	3
chr1	933387	933389	cg15882305				sig-DMD_2	DMV_6	nontrip_trip,normal_nontrip,normal_trip	3
chr1	933684	933686	cg15713103				sig-DMD_2	DMV_6	nontrip_trip,normal_nontrip,normal_trip	3
chr1	933986	933988	cg26003967				sig-DMD_2	DMV_6	nontrip_trip,normal_nontrip,normal_trip	3
chr1	1209195	1209197	cg02581421	5'UTR	NM_194315	UBE2J2	sig-DMD_3	DMV_7	normal_nontrip,normal_trip	2
chr1	1209672	1209674	cg03322129	TSS1500	NM_194458	UBE2J2	sig-DMD_3	DMV_7	normal_nontrip,normal_trip	2
chr1	1209695	1209697	cg01794932	TSS1500	NM_194458	UBE2J2	sig-DMD_3	DMV_7	normal_nontrip,normal_trip	2
chr1	1209719	1209721	cg26163375	TSS1500	NM_194458	UBE2J2	sig-DMD_3	DMV_7	normal_nontrip,normal_trip	2


### Among 4375 signatures, only 3054 regions have HM450k probes

In [16]:
! cut -f8 metadata/sig-DMD.union.includeERR.hm450probes | uniq | wc -l

3054


#### 144/149 ERR signatures have probes

In [17]:
! grep "ers_err" metadata/sig-DMD.union.includeERR.hm450probes | cut -f8 | uniq | wc -l

144


In [18]:
%%bash
echo -e Class"\t"No. signatures"\t" No. signatures have HM450k probes"\t"Percentage >\
metadata/sig-DMD.union.includeERR.hm450probes.count

class="err trip nontrip normal"

for c in $class; do \
    if [ "$c" = "err" ]; then
        exp="$c"
    elif [ "$c" = "trip" ]; then
        exp="_trip[[:print:]]*_trip"
    else
        exp="$c""[[:print:]]*""$c"
    fi;
    countp=$(grep "$exp" metadata/sig-DMD.union.includeERR.hm450probes | \
    cut -f8 | uniq | wc -l | awk '{print $1}');
    count=$(grep -c "$exp" preliminary/sig-DMD.union.includeERR.reference);
    percent=$(echo $countp / $count | bc -l);
    echo -e $c"\t"$count"\t"$countp"\t"$percent >> metadata/sig-DMD.union.includeERR.hm450probes.count;
done
totalp=$(cut -f8 metadata/sig-DMD.union.includeERR.hm450probes | uniq | wc -l)
total=$(wc -l preliminary/sig-DMD.union.includeERR.reference | awk '{print $1}')
percent=$(echo $totalp / $total | bc -l)
echo -e total"\t"$total"\t"$totalp"\t"$percent >> metadata/sig-DMD.union.includeERR.hm450probes.count

In [19]:
! cat metadata/sig-DMD.union.includeERR.hm450probes.count

Class	No. signatures	 No. signatures have HM450k probes	Percentage
err	149	144	.96644295302013422818
trip	950	891	.93789473684210526315
nontrip	926	897	.96868250539956803455
normal	3286	2043	.62172854534388314059
total	4375	3054	.69805714285714285714
